In [1]:
# Import necessary libraries
import yfinance as yf  
import pandas as pd  
from prophet import Prophet
from prophet.plot import plot_plotly
import logging

# Setting up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logging.info("Dependencies imported successfully.")
# function that fetches historical stock data for the specified ticker from Yahoo Finance:
# The function takes the stock symbol (ticker) as input and returns a pandas DataFrame containing the historical stock data.
# The Parameters are:
# ticker (str): The stock symbol for which to download the data.
# start_date (str): The start date for the data download (format: YYYY-MM-DD).
# end_date (str): The end date for the data download (format: YYYY-MM-DD).
# The Returns are:
# pandas.DataFrame: 

def fetch_data(ticker, start_date="2019-01-01", end_date="2024-04-18"):
    try:
        data = yf.download(ticker, start=start_date, end=end_date)
        logging.info("Data downloaded successfully.")
        return data
    except Exception as e:
        logging.error(f"Failed to download data: {e}")
        return pd.DataFrame()
# The function prepares the stock data for use with the Prophet library
# by selecting the 'Date' and 'Close' columns and renaming them to 'ds' and 'y', respectively.
# Returns a DataFrame suitable for use with Prophet modeling
def prepare_data_for_prophet(data):
    prepped_df = data.reset_index()[['Date', 'Close']].rename(columns={'Date': 'ds', 'Close': 'y'})
    logging.info("Data prepared for Prophet.")
    return prepped_df
# The function trains a Prophet model on the provided data.
def train_prophet_model(prepped_df):
    model = Prophet(daily_seasonality=False, yearly_seasonality=True)
    model.fit(prepped_df)
    logging.info("Prophet model trained successfully.")
    return model
# The function uses the trained Prophet model to make future predictions.
# The Parameters are: periods (int) - Number of days to predict into the future. 
"""Adjust the periods parameter to specify the number of days into the future for which you want to make predictions."""
def make_forecast(model, periods=365):
    future = model.make_future_dataframe(periods=periods)
    forecast = model.predict(future)
    logging.info("Forecast made for future periods.")
    return forecast
# The function plots the forecasted data using Prophet's plot functionality.
def plot_forecast(model, forecast):
    fig = plot_plotly(model, forecast)  # This returns a plotly Figure
    fig.show()
# The fubctions runs the complete workflow.
def run_prophet_forecasting(ticker):
    data = fetch_data(ticker)
    if not data.empty:
        prepared_data = prepare_data_for_prophet(data)
        model = train_prophet_model(prepared_data)
        forecast = make_forecast(model, periods=365)
        plot_forecast(model, forecast)
        return forecast
    else:
        logging.error("No data available to process.")
        return pd.DataFrame()

# The function take the stock symbol (ticker) as input and starts the forecasting process, by calling the run_prophet_forecasting function.
"""User should enter in the stock ticker symbol for which they want to make predictions."""
ticker = 'IBM'
forecast_results = run_prophet_forecasting(ticker)

#save the forecast results to a CSV file
if not forecast_results.empty:
    filepath = r'C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\MCP_ibm_aapl_data\McP_data_files\forecast_results_IBM.csv'
    forecast_results.to_csv(filepath, index=False)
    logging.info(f"Forecast results saved to: {filepath}")
else:
    logging.error("Forecast results DataFrame is empty, nothing was saved.")


2024-04-21 10:30:46,524 - INFO - Dependencies imported successfully.
[*********************100%%**********************]  1 of 1 completed
2024-04-21 10:30:48,332 - INFO - Data downloaded successfully.
2024-04-21 10:30:48,334 - INFO - Data prepared for Prophet.
2024-04-21 10:30:48,800 - DEBUG - cmd: where.exe tbb.dll
cwd: None
2024-04-21 10:30:48,874 - DEBUG - Adding TBB (C:\Users\mikea\AppData\Roaming\Python\Python311\site-packages\prophet\stan_model\cmdstan-2.33.1\stan\lib\stan_math\lib\tbb) to PATH
2024-04-21 10:30:48,911 - DEBUG - input tempfile: C:\Users\mikea\AppData\Local\Temp\tmp6ijh01_x\ne7uiv5l.json
2024-04-21 10:30:49,009 - DEBUG - input tempfile: C:\Users\mikea\AppData\Local\Temp\tmp6ijh01_x\vw4n_5w3.json
2024-04-21 10:30:49,014 - DEBUG - idx 0
2024-04-21 10:30:49,014 - DEBUG - running CmdStan, num_threads: None
2024-04-21 10:30:49,014 - DEBUG - CmdStan args: ['C:\\Users\\mikea\\AppData\\Roaming\\Python\\Python311\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'ran

2024-04-21 10:30:50,724 - INFO - Forecast results saved to: C:\Users\mikea\Desktop\Dataclass2\StockPrediction_Project3\Index_Fund_AI_Anaysis_App_Prj_3\MCP_ibm_aapl_data\McP_data_files\forecast_results_IBM.csv


In [2]:
# Check dataframe 
forecast_results.head()


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2019-01-02,118.734058,114.227785,130.136490,118.734058,118.734058,2.895686,2.895686,2.895686,1.910086,1.910086,1.910086,0.985600,0.985600,0.985600,0.0,0.0,0.0,121.629744
1,2019-01-03,118.819730,113.777178,129.548908,118.819730,118.819730,2.835698,2.835698,2.835698,1.820278,1.820278,1.820278,1.015420,1.015420,1.015420,0.0,0.0,0.0,121.655428
2,2019-01-04,118.905402,113.883027,129.265709,118.905402,118.905402,2.891864,2.891864,2.891864,1.837329,1.837329,1.837329,1.054535,1.054535,1.054535,0.0,0.0,0.0,121.797266
3,2019-01-07,119.162418,114.651807,130.267866,119.162418,119.162418,3.149398,3.149398,3.149398,1.904179,1.904179,1.904179,1.245219,1.245219,1.245219,0.0,0.0,0.0,122.311816
4,2019-01-08,119.248090,115.181281,130.029710,119.248090,119.248090,3.280234,3.280234,3.280234,1.941818,1.941818,1.941818,1.338416,1.338416,1.338416,0.0,0.0,0.0,122.528324
